# Focus Buddy Agent - Concierge System for Deep Work

**Track:** Concierge Agents

**Problem:** People struggle with decision fatigue and context switching when facing messy to-do lists.

**Solution:** Multi-agent system with Planner + Coach agents that creates realistic 25-30 minute focus plans.

**Concepts:** Multi-agent roles, Tools/function calling, Session memory

## Setup & Installation

In [ ]:
!pip install -q google-generativeai

In [ ]:
import google.generativeai as genai
import os
from dataclasses import dataclass
from typing import List
import re

# Configure Gemini API
# In Kaggle: Add GEMINI_API_KEY as a secret in Settings
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

## Tool Definitions

Three core tools for task processing:

In [ ]:
@dataclass
class Task:
    title: str
    deadline: str = ""
    estimated_minutes: int = 10

@dataclass
class ScheduledBlock:
    start_minute: int
    end_minute: int
    task_title: str

def parse_tasks(raw_text: str) -> List[Task]:
    """Parse messy task text into structured Task objects."""
    tasks = []
    lines = [line.strip() for line in raw_text.split('\n') if line.strip()]
    
    for line in lines:
        clean_line = re.sub(r'^[-*•\d.)\]]+\s*', '', line)
        
        # Extract time estimates
        time_match = re.search(r'(\d+)\s*(min|hour|hr)', clean_line, re.IGNORECASE)
        estimated_minutes = 10
        if time_match:
            num = int(time_match.group(1))
            unit = time_match.group(2).lower()
            estimated_minutes = num if 'min' in unit else num * 60
            clean_line = re.sub(r'\d+\s*(min|hour|hr)', '', clean_line, flags=re.IGNORECASE)
        
        # Extract deadline
        deadline_match = re.search(r'(due|deadline|by)\s*:?\s*(\S+)', clean_line, re.IGNORECASE)
        deadline = deadline_match.group(2) if deadline_match else ""
        if deadline_match:
            clean_line = re.sub(r'(due|deadline|by)\s*:?\s*\S+', '', clean_line, flags=re.IGNORECASE)
        
        clean_line = clean_line.strip(' ,-:')
        if clean_line:
            tasks.append(Task(title=clean_line, deadline=deadline, estimated_minutes=estimated_minutes))
    
    return tasks

def prioritize_tasks(tasks: List[Task]) -> List[Task]:
    """Sort tasks by urgency (deadline) then by time."""
    def priority_key(task):
        has_deadline = 1 if task.deadline else 0
        return (-has_deadline, task.estimated_minutes)
    
    return sorted(tasks, key=priority_key)

def create_focus_schedule(tasks: List[Task], available_minutes: int) -> List[ScheduledBlock]:
    """Create schedule fitting within available time."""
    schedule = []
    current_minute = 0
    
    for task in tasks:
        if current_minute + task.estimated_minutes <= available_minutes:
            schedule.append(ScheduledBlock(
                start_minute=current_minute,
                end_minute=current_minute + task.estimated_minutes,
                task_title=task.title
            ))
            current_minute += task.estimated_minutes
        else:
            remaining = available_minutes - current_minute
            if remaining >= 5:
                schedule.append(ScheduledBlock(
                    start_minute=current_minute,
                    end_minute=available_minutes,
                    task_title=f"{task.title} (partial)"
                ))
            break
    
    return schedule

print("✅ Tools defined successfully")

## Agent System Prompt

Multi-agent specification with Planner and Coach roles:

In [ ]:
AGENT_SPEC = """You are *Focus Buddy*, a concierge AI agent that helps users turn messy to-do lists into realistic 25–30 minute deep-work sprint plans.

## Multi-Agent Roles
1. **Planner Agent** – Parse tasks, prioritize, create schedule
2. **Coach Agent** – Motivate, confirm plan, facilitate check-ins

## Workflow
1. Understand time window (default 25 min) and energy level
2. Parse tasks into structured format
3. Prioritize by urgency, importance, time
4. Create realistic schedule fitting time window
5. Present plan with clear timeline
6. Set check-in expectations

## Output Format
1. **Summary** – one sentence focus statement
2. **Timeline** – bullet list with time ranges
3. **Checklist** – 3-5 actionable items
4. **Check-in** – when to return

Be concise, practical, and encouraging. Reduce task count if user seems overwhelmed.
"""

## Main Agent Function

Orchestrates the multi-agent workflow:

In [ ]:
# Session memory
session_memory = {
    "current_focus_plan": None,
    "tasks": [],
    "completed": []
}

def run_focus_buddy(user_input: str, available_minutes: int = 25):
    """Main agent loop - multi-agent orchestration."""
    
    print("🤖 PLANNER AGENT ACTIVE")
    print("="*50)
    
    # Step 1: Parse tasks
    print("🔍 Parsing your tasks...")
    tasks = parse_tasks(user_input)
    session_memory["tasks"] = tasks
    print(f"   Found {len(tasks)} tasks")
    for i, task in enumerate(tasks, 1):
        print(f"   {i}. {task.title} ({task.estimated_minutes} min)")
    
    # Step 2: Prioritize
    print("\n📊 Prioritizing...")
    prioritized = prioritize_tasks(tasks)
    print(f"   Top priority: {prioritized[0].title}")
    
    # Step 3: Create schedule
    print(f"\n📅 Creating {available_minutes}-minute focus plan...")
    schedule = create_focus_schedule(prioritized, available_minutes)
    session_memory["current_focus_plan"] = schedule
    print(f"   Scheduled {len(schedule)} blocks\n")
    
    print("🎯 COACH AGENT ACTIVE")
    print("="*50)
    
    # Step 4: Format with Gemini
    model = genai.GenerativeModel('gemini-2.0-flash-exp')
    
    schedule_text = "\n".join([
        f"  {block.start_minute}-{block.end_minute} min: {block.task_title}"
        for block in schedule
    ])
    
    prompt = f"""{AGENT_SPEC}

The user provided these tasks:
{user_input}

I've created this {available_minutes}-minute schedule:
{schedule_text}

Now present this plan following the output format: Summary, Timeline, Checklist, Check-in. Be encouraging."""

    response = model.generate_content(prompt)
    return response.text

## Example 1: Standard Work Session

In [ ]:
example_1 = """
- Review pull requests (20 min)
- Write unit tests for auth module
- Update documentation due: today
- Reply to 3 urgent emails (15 min)
- Plan sprint retrospective
"""

print("📋 USER INPUT:")
print(example_1)
print("\n" + "="*60 + "\n")

result_1 = run_focus_buddy(example_1, available_minutes=30)
print("\n" + result_1)

## Example 2: Overwhelmed User (Agent reduces scope)

In [ ]:
example_2 = """
I have so much to do:
- Finish project proposal 2 hours
- Read 50 pages for tomorrow's class 1 hour
- Debug production issue
- Call mom
- Prep presentation slides 90min
- Grocery shopping
- Exercise
"""

print("📋 USER INPUT (Overwhelmed):")
print(example_2)
print("\n" + "="*60 + "\n")

result_2 = run_focus_buddy(example_2, available_minutes=25)
print("\n" + result_2)

## Demonstration of Session Memory

In [ ]:
print("📊 SESSION MEMORY STATE:")
print(f"\nTotal tasks tracked: {len(session_memory['tasks'])}")
print(f"Current focus plan has {len(session_memory['current_focus_plan'])} blocks")
print("\nScheduled blocks:")
for block in session_memory['current_focus_plan']:
    print(f"  • {block.start_minute}-{block.end_minute} min: {block.task_title}")

## Summary

**Concepts Demonstrated:**
1. ✅ **Multi-agent system** - Planner and Coach agents with distinct roles
2. ✅ **Tools/function calling** - parse_tasks, prioritize_tasks, create_focus_schedule
3. ✅ **Session memory** - Tracks tasks and focus plans across interactions

**Value:**
- Reduces decision fatigue
- Creates realistic, time-bounded plans
- Prevents over-commitment
- Enables repeatable focus cycles